#### Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
import catboost as ct
from sklearn.metrics import log_loss, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import optuna
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier
import numpy as np

#### Loading data

In [2]:
train = pd.read_csv(r'C:\Users\formy\Downloads\icr-identify-age-related-conditions\train.csv')
test = pd.read_csv(r'C:\Users\formy\Downloads\icr-identify-age-related-conditions\test.csv')
greeks = pd.read_csv(r'C:\Users\formy\Downloads\icr-identify-age-related-conditions\greeks.csv')
sample_submission = pd.read_csv(r'C:\Users\formy\Downloads\icr-identify-age-related-conditions\sample_submission.csv')

##---- Copy from here

df = pd.merge(train, greeks, how='left', on='Id')
def encode(dataframe):
    le = LabelEncoder()
    obj = list(dataframe.loc[:, dataframe.dtypes == 'object'].columns)
    for i in obj:
        if i not in ['id', 'Epsilon']:
            dataframe[i] = le.fit_transform(dataframe[i])
    return dataframe
imputer = KNNImputer(n_neighbors=2)

features = ['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL']

df = encode(df)  
test = encode(test)

df[features] = imputer.fit_transform(df[features])
test[features] = imputer.fit_transform(test[features])

target = 'Class'
X = df[features]
y = df[target]

from sklearn.model_selection import train_test_split
# Assuming X and y are your features and labels
X_t, X_test, y_t, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Tuning

In [3]:
def balanced_log_loss(y_true, y_pred, class_weights):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    w0 = class_weights[0]
    w1 = class_weights[1]
    loss = -2*(w1 * y_true * np.log(y_pred[:,1]) + w0 * (1 - y_true) * np.log(y_pred[:,0]))/(w0+w1)
    return loss.mean()

#### Final Model

In [5]:
best_params = {'iterations': 212,
 'depth': 4,
 'learning_rate': 0.02995163795315085,
 'random_strength': 0,
 'bagging_temperature': 0.010387485768987258,
 'od_type': 'Iter',
 'od_wait': 37}
class_counts = y.value_counts()
class_weights = {class_counts.index[0]: class_counts[1] / (class_counts[1]+class_counts[0]),
                     class_counts.index[1]: class_counts[0] / (class_counts[1]+class_counts[0])}
final_model = CatBoostClassifier(**best_params, loss_function='Logloss', eval_metric='Logloss',task_type = 'GPU', class_weights=class_weights)
final_model.fit(X, y,verbose = 0) # Train on the full data
prediction = final_model.predict_proba(test[features])

In [31]:
import random 
# define a StratifiedKFold object
strat_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=int(random.random()*1000))

# list to store log loss for each fold
log_loss_scores = []

for train_index, test_index in strat_kfold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model = CatBoostClassifier(**best_params, loss_function='Logloss', eval_metric='Logloss', class_weights=class_weights)
    stacking_model.fit(X_train, y_train)
    
    y_pred = stacking_model.predict_proba(X_test)
    fold_log_loss = balanced_log_loss(y_test, y_pred,class_weights)
    log_loss_scores.append(fold_log_loss)
    
# print the mean log loss across all folds
print("Mean Log Loss on test data across 10 folds: ", np.mean(log_loss_scores))

0:	learn: 0.6775321	total: 2.92ms	remaining: 58.5ms
1:	learn: 0.6627645	total: 4.33ms	remaining: 41.2ms
2:	learn: 0.6487835	total: 5.55ms	remaining: 33.3ms
3:	learn: 0.6355510	total: 6.75ms	remaining: 28.7ms
4:	learn: 0.6230079	total: 9.93ms	remaining: 31.8ms
5:	learn: 0.6111215	total: 11.2ms	remaining: 28.1ms
6:	learn: 0.5998478	total: 12.8ms	remaining: 25.6ms
7:	learn: 0.5891476	total: 14.4ms	remaining: 23.5ms
8:	learn: 0.5789888	total: 15.6ms	remaining: 20.8ms
9:	learn: 0.5693349	total: 16.9ms	remaining: 18.6ms
10:	learn: 0.5601515	total: 20.4ms	remaining: 18.5ms
11:	learn: 0.5514155	total: 23.1ms	remaining: 17.3ms
12:	learn: 0.5430986	total: 24.3ms	remaining: 14.9ms
13:	learn: 0.5351721	total: 25.5ms	remaining: 12.7ms
14:	learn: 0.5276095	total: 26.6ms	remaining: 10.6ms
15:	learn: 0.5203892	total: 27.6ms	remaining: 8.63ms
16:	learn: 0.5135380	total: 30.7ms	remaining: 7.22ms
17:	learn: 0.5069980	total: 32.7ms	remaining: 5.45ms
18:	learn: 0.5007540	total: 34ms	remaining: 3.58ms
19:	l

0:	learn: 0.6773124	total: 1.43ms	remaining: 28.7ms
1:	learn: 0.6623460	total: 3.42ms	remaining: 32.5ms
2:	learn: 0.6481775	total: 4.55ms	remaining: 27.3ms
3:	learn: 0.6347761	total: 5.72ms	remaining: 24.3ms
4:	learn: 0.6220802	total: 6.87ms	remaining: 22ms
5:	learn: 0.6100422	total: 8.4ms	remaining: 21ms
6:	learn: 0.5986300	total: 9.71ms	remaining: 19.4ms
7:	learn: 0.5878092	total: 12.1ms	remaining: 19.7ms
8:	learn: 0.5775296	total: 13.8ms	remaining: 18.4ms
9:	learn: 0.5677671	total: 15.1ms	remaining: 16.6ms
10:	learn: 0.5584853	total: 16.6ms	remaining: 15.1ms
11:	learn: 0.5496548	total: 17.9ms	remaining: 13.4ms
12:	learn: 0.5412504	total: 19.4ms	remaining: 11.9ms
13:	learn: 0.5332425	total: 20.6ms	remaining: 10.3ms
14:	learn: 0.5256081	total: 21.8ms	remaining: 8.72ms
15:	learn: 0.5183169	total: 23ms	remaining: 7.18ms
16:	learn: 0.5113926	total: 23.9ms	remaining: 5.63ms
17:	learn: 0.5047939	total: 24.8ms	remaining: 4.14ms
18:	learn: 0.4984921	total: 26.3ms	remaining: 2.77ms
19:	learn:

In [30]:
log_loss_scores

[0.2319797196583373,
 0.26568043742012076,
 0.26911297117232647,
 0.3003142819017332,
 0.3099953246405817]

In [ ]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from mlxtend.classifier import StackingCVClassifier

class_counts = y.value_counts()
class_weights = {class_counts.index[0]: class_counts[1] / (class_counts[1]+class_counts[0]),
                     class_counts.index[1]: class_counts[0] / (class_counts[1]+class_counts[0])}
scale_pos_weight = len(y[y == 0]) / len(y[y == 1])

Catboost_params = {'iterations': 212,'depth': 4,'learning_rate': 0.02995163795315085,'random_strength': 0,'bagging_temperature': 0.010387485768987258,'od_type': 'Iter','od_wait': 37}
xgboost_params = {'n_estimators': 609, 'max_depth': 5, 'learning_rate': 0.024691789283433796, 'subsample': 0.6145800172695441,
                  'colsample_bytree': 0.9890624757789978, 'gamma': 0.6445547957115808, 'reg_lambda': 3.7723285451952338}Catboost_params = {'iterations': 212,'depth': 4,'learning_rate': 0.02995163795315085,'random_strength': 0,'bagging_temperature': 0.010387485768987258,'od_type': 'Iter','od_wait': 37}
xgboost_params = {'n_estimators': 609, 'max_depth': 5, 'learning_rate': 0.024691789283433796, 'subsample': 0.6145800172695441,
                  'colsample_bytree': 0.9890624757789978, 'gamma': 0.6445547957115808, 'reg_lambda': 3.7723285451952338}
meta_params = {'iterations': 21,
 'depth': 3,
 'learning_rate': 0.02995163795315085,
 'random_strength': 0,
 'bagging_temperature': 0.010387485768987258,
 'od_type': 'Iter',
 'od_wait': 37}

cat = CatBoostClassifier(**Catboost_params, loss_function='Logloss', eval_metric='Logloss',class_weights = class_weights,verbose=False,task_type = 'GPU')
xgb = XGBClassifier(**xgboost_params,scale_pos_weight=scale_pos_weight,tree_method='gpu_hist' )
meta_model = CatBoostClassifier(**meta_params, loss_function='Logloss', eval_metric='Logloss',class_weights = class_weights)

stacking_model = StackingCVClassifier(classifiers=[cat, xgb],
                                      meta_classifier=meta_model,
                                      random_state=0)

stacking_model.fit(X_train, y_train)
predictions = stacking_model.predict(X_test)

In [ ]:
import tensorflow as tf
print(tf.test.is_gpu_available())

In [ ]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import log_loss
import optuna

def objective(trial):
    # Parameters to tune
    params = {
    'iterations': trial.suggest_int('iterations', 250,700),
    'depth': trial.suggest_int('depth', 5,14),
    'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.03),
    'random_strength': trial.suggest_int('random_strength', 0, 100),
    'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 90.00),
    'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
    'od_wait': trial.suggest_int('od_wait', 10, 50),
    'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
    'border_count': trial.suggest_int('border_count', 32, 255),
    'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 10),
    'leaf_estimation_method': trial.suggest_categorical('leaf_estimation_method', ['Newton', 'Gradient']),
    'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian']),
    }
    
    # Compute class weights
    class_counts = y.value_counts()
    class_weights = {class_counts.index[0]: class_counts[1] / (class_counts[1]+class_counts[0]),
                     class_counts.index[1]: class_counts[0] / (class_counts[1]+class_counts[0])}

    n_splits = 10
    n_repeats = 10  # Number of times cross-validator needs to be repeated.
    rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)
    
    log_loss_scores = []
    for train_index, valid_index in rskf.split(X, y):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        model = CatBoostClassifier(**params, loss_function='Logloss', eval_metric='Logloss', task_type='GPU', class_weights=class_weights)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100, verbose=False)
        
        pred_valid = model.predict_proba(X_valid)
        log_loss_scores.append(balanced_log_loss(y_valid, pred_valid,class_weights))
    
    return np.mean(log_loss_scores)

# Create the study and run optimization
#study = optuna.create_study(direction='minimize',study_name='catboost_study_ICR_Kaggle')
study.optimize(objective, n_trials=40)

#### XGBoost instead of CATBoost


In [39]:
scale_pos_weight = len(y[y == 0]) / len(y[y == 1])

import xgboost as xgb

def objective(trial):
    # Parameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 350,700),
        'max_depth': trial.suggest_int('max_depth', 5,8),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.02, 0.03),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1),
        'gamma': trial.suggest_uniform('gamma', 0, 1),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10),
    }
    
    # Compute class weights
    class_counts = y.value_counts()
    
    class_weights = {class_counts.index[0]: class_counts[1] / (class_counts[1]+class_counts[0]),
                     class_counts.index[1]: class_counts[0] / (class_counts[1]+class_counts[0])}

    n_splits = 10
    n_repeats = 10  # Number of times cross-validator needs to be repeated.
    rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)
    
    log_loss_scores = []
    for train_index, valid_index in rskf.split(X, y):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        model = xgb.XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss',scale_pos_weight=scale_pos_weight)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100, verbose=False)
        
        pred_valid = model.predict_proba(X_valid)
        log_loss_scores.append(balanced_log_loss(y_valid, pred_valid,class_weights))
    
    return np.mean(log_loss_scores)

# Create the study and run optimization
study = optuna.create_study(direction='minimize', study_name='xgboost_study_ICR_Kaggle')
study.optimize(objective, n_trials=40)

[I 2023-05-29 16:46:59,389] A new study created in memory with name: xgboost_study_ICR_Kaggle
[I 2023-05-29 16:53:00,572] Trial 0 finished with value: 0.16765302845523414 and parameters: {'n_estimators': 465, 'max_depth': 7, 'learning_rate': 0.023688619344923514, 'subsample': 0.537348864755476, 'colsample_bytree': 0.9082555864860452, 'gamma': 0.26978878445388976, 'reg_lambda': 0.9762176099025719}. Best is trial 0 with value: 0.16765302845523414.
[I 2023-05-29 16:58:27,871] Trial 1 finished with value: 0.17399680008652937 and parameters: {'n_estimators': 551, 'max_depth': 8, 'learning_rate': 0.02957378313942339, 'subsample': 0.6271870395473771, 'colsample_bytree': 0.5522796065760083, 'gamma': 0.16713518502638747, 'reg_lambda': 0.10929123166981597}. Best is trial 0 with value: 0.16765302845523414.
[I 2023-05-29 17:04:04,783] Trial 2 finished with value: 0.17303224606044537 and parameters: {'n_estimators': 415, 'max_depth': 7, 'learning_rate': 0.02482667299261517, 'subsample': 0.716532633

[I 2023-05-29 19:06:57,490] Trial 22 finished with value: 0.1630392546608995 and parameters: {'n_estimators': 618, 'max_depth': 5, 'learning_rate': 0.022925120902596473, 'subsample': 0.6569198977104352, 'colsample_bytree': 0.992972746545483, 'gamma': 0.3370937406360785, 'reg_lambda': 3.371506262436372}. Best is trial 13 with value: 0.1593170862727116.
[I 2023-05-29 19:15:18,699] Trial 23 finished with value: 0.16199118888854247 and parameters: {'n_estimators': 664, 'max_depth': 6, 'learning_rate': 0.024401929210290732, 'subsample': 0.6013010442889767, 'colsample_bytree': 0.9559032644724417, 'gamma': 0.5027505210668028, 'reg_lambda': 3.922405956150893}. Best is trial 13 with value: 0.1593170862727116.
[I 2023-05-29 19:22:37,669] Trial 24 finished with value: 0.1606694007215848 and parameters: {'n_estimators': 700, 'max_depth': 5, 'learning_rate': 0.025479307756414046, 'subsample': 0.5648339858563659, 'colsample_bytree': 0.8948542911042681, 'gamma': 0.5635003048802595, 'reg_lambda': 1.36

#### Optimize Hyperparameters of Stacking Model

In [37]:
def objective(trial):
    # Parameters to tune
    Catboost_params = {'iterations': 212,'depth': 4,'learning_rate': 0.02995163795315085,'random_strength': 0,'bagging_temperature': 0.010387485768987258,'od_type': 'Iter','od_wait': 37}
    xgboost_params = {'n_estimators': 609, 'max_depth': 5, 'learning_rate': 0.024691789283433796, 'subsample': 0.6145800172695441,
                  'colsample_bytree': 0.9890624757789978, 'gamma': 0.6445547957115808, 'reg_lambda': 3.7723285451952338}
    meta_params = {
        'iterations': trial.suggest_int('iterations', 10, 100),
        'depth': trial.suggest_int('depth', 1, 3),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.2),
        'random_strength': trial.suggest_int('random_strength', 0, 100),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 90.00),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait': trial.suggest_int('od_wait', 10, 50)
    }

    cat = CatBoostClassifier(**Catboost_params, loss_function='Logloss', eval_metric='Logloss',class_weights=class_weights,verbose=False,task_type='GPU')
    xgb = XGBClassifier(**xgboost_params,scale_pos_weight=scale_pos_weight,tree_method='gpu_hist' )
    meta_model = CatBoostClassifier(**meta_params, loss_function='Logloss', eval_metric='Logloss',class_weights = class_weights)

    stacking_model = StackingCVClassifier(classifiers=[cat, xgb],
                                          meta_classifier=meta_model,
                                          random_state=0)

    # Perform cross-validation and compute log loss
    log_loss_scores = []
    for train_index, valid_index in StratifiedKFold(n_splits=3).split(X, y):
        X_train_cv, X_valid_cv = X.iloc[train_index], X.iloc[valid_index]
        y_train_cv, y_valid_cv = y.iloc[train_index], y.iloc[valid_index]
        stacking_model.fit(X_train_cv, y_train_cv)
        pred_valid = stacking_model.predict_proba(X_valid_cv)
        log_loss_scores.append(balanced_log_loss(y_valid_cv, pred_valid, class_weights))
        
    return np.mean(log_loss_scores)

study = optuna.create_study(direction='minimize',study_name = 'stack_study')  # We aim to minimize log loss
study.optimize(objective, n_trials=10)

[I 2023-05-30 19:58:46,401] A new study created in memory with name: stack_study


0:	learn: 0.6876279	total: 1.29ms	remaining: 108ms
1:	learn: 0.6822190	total: 2.45ms	remaining: 101ms
2:	learn: 0.6772073	total: 3.3ms	remaining: 89ms
3:	learn: 0.6719996	total: 4.07ms	remaining: 81.3ms
4:	learn: 0.6671766	total: 5.06ms	remaining: 80ms
5:	learn: 0.6624436	total: 6.46ms	remaining: 84ms
6:	learn: 0.6578092	total: 7.24ms	remaining: 79.6ms
7:	learn: 0.6532643	total: 8.07ms	remaining: 76.7ms
8:	learn: 0.6488140	total: 8.93ms	remaining: 74.4ms
9:	learn: 0.6444477	total: 9.67ms	remaining: 71.5ms
10:	learn: 0.6401688	total: 10.4ms	remaining: 69.1ms
11:	learn: 0.6359735	total: 11.2ms	remaining: 67.1ms
12:	learn: 0.6318590	total: 11.8ms	remaining: 64.7ms
13:	learn: 0.6278320	total: 12.5ms	remaining: 62.5ms
14:	learn: 0.6238754	total: 13.2ms	remaining: 60.6ms
15:	learn: 0.6196374	total: 13.9ms	remaining: 58.9ms
16:	learn: 0.6158319	total: 15ms	remaining: 59.1ms
17:	learn: 0.6120957	total: 15.8ms	remaining: 57.9ms
18:	learn: 0.6080754	total: 16.5ms	remaining: 56.3ms
19:	learn: 0.6

[I 2023-05-30 20:00:14,311] Trial 0 finished with value: 0.25855881282770504 and parameters: {'iterations': 84, 'depth': 1, 'learning_rate': 0.011009989779023439, 'random_strength': 45, 'bagging_temperature': 0.03928979271242604, 'od_type': 'IncToDec', 'od_wait': 41}. Best is trial 0 with value: 0.25855881282770504.


0:	learn: 0.6887260	total: 978us	remaining: 81.2ms
1:	learn: 0.6843911	total: 2.68ms	remaining: 110ms
2:	learn: 0.6800943	total: 3.47ms	remaining: 93.8ms
3:	learn: 0.6759204	total: 4.21ms	remaining: 84.1ms
4:	learn: 0.6717737	total: 4.96ms	remaining: 78.3ms
5:	learn: 0.6677184	total: 5.69ms	remaining: 74ms
6:	learn: 0.6637388	total: 6.73ms	remaining: 74ms
7:	learn: 0.6598406	total: 7.5ms	remaining: 71.2ms
8:	learn: 0.6560173	total: 8.86ms	remaining: 73.8ms
9:	learn: 0.6522703	total: 9.69ms	remaining: 71.7ms
10:	learn: 0.6485964	total: 10.7ms	remaining: 71.1ms
11:	learn: 0.6449924	total: 11.4ms	remaining: 68.7ms
12:	learn: 0.6414614	total: 12.2ms	remaining: 66.7ms
13:	learn: 0.6379977	total: 13.1ms	remaining: 65.7ms
14:	learn: 0.6346000	total: 13.9ms	remaining: 63.9ms
15:	learn: 0.6311556	total: 14.9ms	remaining: 63.5ms
16:	learn: 0.6278797	total: 15.7ms	remaining: 61.8ms
17:	learn: 0.6246634	total: 16.5ms	remaining: 60.4ms
18:	learn: 0.6213832	total: 17.8ms	remaining: 60.9ms
19:	learn:

[I 2023-05-30 20:01:37,755] Trial 1 finished with value: 0.1990954643292123 and parameters: {'iterations': 37, 'depth': 2, 'learning_rate': 0.14208720838681166, 'random_strength': 32, 'bagging_temperature': 4.361497178814604, 'od_type': 'IncToDec', 'od_wait': 31}. Best is trial 1 with value: 0.1990954643292123.


...

0:	learn: 0.6237734	total: 977us	remaining: 62.6ms
1:	learn: 0.5759617	total: 3.31ms	remaining: 104ms
2:	learn: 0.5351845	total: 4.16ms	remaining: 85.9ms
3:	learn: 0.5094182	total: 4.93ms	remaining: 75.2ms
4:	learn: 0.4855708	total: 5.71ms	remaining: 68.6ms
5:	learn: 0.4679743	total: 6.48ms	remaining: 63.7ms
6:	learn: 0.4549370	total: 7.19ms	remaining: 59.6ms
7:	learn: 0.4451419	total: 7.83ms	remaining: 55.8ms
8:	learn: 0.4378278	total: 8.59ms	remaining: 53.4ms
9:	learn: 0.4323810	total: 9.33ms	remaining: 51.3ms
10:	learn: 0.4282196	total: 10.1ms	remaining: 49.5ms
11:	learn: 0.4251585	total: 10.8ms	remaining: 47.8ms
12:	learn: 0.4228291	total: 11.6ms	remaining: 46.5ms
13:	learn: 0.4210613	total: 12.4ms	remaining: 45.2ms
14:	learn: 0.4197292	total: 13.1ms	remaining: 43.8ms
15:	learn: 0.4169399	total: 13.9ms	remaining: 42.6ms
16:	learn: 0.4162108	total: 14.7ms	remaining: 41.4ms
17:	learn: 0.4156693	total: 15.4ms	remaining: 40.2ms
18:	learn: 0.4139062	total: 16.2ms	remaining: 39.2ms
19:	l

[I 2023-05-30 20:13:25,252] Trial 8 finished with value: 0.20174503532594804 and parameters: {'iterations': 48, 'depth': 2, 'learning_rate': 0.06067234832182191, 'random_strength': 52, 'bagging_temperature': 0.0729207304182486, 'od_type': 'Iter', 'od_wait': 45}. Best is trial 7 with value: 0.19857922178215515.


0:	learn: 0.6658645	total: 1.21ms	remaining: 56.7ms
1:	learn: 0.6414417	total: 2.3ms	remaining: 52.9ms
2:	learn: 0.6224169	total: 3.27ms	remaining: 49.1ms
3:	learn: 0.6024867	total: 4.19ms	remaining: 46.1ms
4:	learn: 0.5845275	total: 5.17ms	remaining: 44.4ms
5:	learn: 0.5702481	total: 6.07ms	remaining: 42.5ms
6:	learn: 0.5573629	total: 6.94ms	remaining: 40.7ms
7:	learn: 0.5456908	total: 8.29ms	remaining: 41.5ms
8:	learn: 0.5330964	total: 9.62ms	remaining: 41.7ms
9:	learn: 0.5216450	total: 10.6ms	remaining: 40.3ms
10:	learn: 0.5112099	total: 11.7ms	remaining: 39.5ms
11:	learn: 0.5032265	total: 12.7ms	remaining: 38.1ms
12:	learn: 0.4956609	total: 13.7ms	remaining: 37ms
13:	learn: 0.4875527	total: 14.6ms	remaining: 35.5ms
14:	learn: 0.4814112	total: 15.6ms	remaining: 34.3ms
15:	learn: 0.4757995	total: 16.6ms	remaining: 33.2ms
16:	learn: 0.4694152	total: 17.6ms	remaining: 32.1ms
17:	learn: 0.4635729	total: 18.5ms	remaining: 30.8ms
18:	learn: 0.4582290	total: 19.4ms	remaining: 29.6ms
19:	le

[I 2023-05-30 20:14:58,658] Trial 9 finished with value: 0.20843568708318763 and parameters: {'iterations': 42, 'depth': 2, 'learning_rate': 0.05147521358942731, 'random_strength': 62, 'bagging_temperature': 1.420844005913833, 'od_type': 'Iter', 'od_wait': 25}. Best is trial 7 with value: 0.19857922178215515.


0:	learn: 0.6699027	total: 2.46ms	remaining: 101ms
1:	learn: 0.6487372	total: 4.17ms	remaining: 83.3ms
2:	learn: 0.6319564	total: 5.36ms	remaining: 69.7ms
3:	learn: 0.6141456	total: 6.28ms	remaining: 59.7ms
4:	learn: 0.5978590	total: 7.54ms	remaining: 55.8ms
5:	learn: 0.5847346	total: 9.38ms	remaining: 56.3ms
6:	learn: 0.5727309	total: 11.6ms	remaining: 58ms
7:	learn: 0.5617221	total: 12.3ms	remaining: 52.4ms
8:	learn: 0.5497153	total: 13.1ms	remaining: 48ms
9:	learn: 0.5386427	total: 14ms	remaining: 44.9ms
10:	learn: 0.5284694	total: 15.2ms	remaining: 42.9ms
11:	learn: 0.5205317	total: 16.2ms	remaining: 40.4ms
12:	learn: 0.5129286	total: 17ms	remaining: 38ms
13:	learn: 0.5047069	total: 17.9ms	remaining: 35.8ms
14:	learn: 0.4983463	total: 19.8ms	remaining: 35.6ms
15:	learn: 0.4924797	total: 20.7ms	remaining: 33.6ms
16:	learn: 0.4858134	total: 21.6ms	remaining: 31.8ms
17:	learn: 0.4796265	total: 22.8ms	remaining: 30.3ms
18:	learn: 0.4738942	total: 23.7ms	remaining: 28.7ms
19:	learn: 0.4

In [36]:
study.best_params

{'iterations': 124,
 'depth': 8,
 'learning_rate': 0.11450496405941944,
 'random_strength': 81,
 'bagging_temperature': 0.01845675623183653,
 'od_type': 'Iter',
 'od_wait': 23,
 'n_estimators': 124,
 'max_depth': 9,
 'subsample': 0.7336679976081282,
 'colsample_bytree': 0.9646503180181017,
 'gamma': 0.5295884569975454,
 'reg_lambda': 3.1179201560121785}

In [16]:
# Compute class weights
import random
losses = []
class_counts = y.value_counts()
class_weights = {class_counts.index[0]: class_counts[1] / (class_counts[1]+class_counts[0]),
                     class_counts.index[1]: class_counts[0] / (class_counts[1]+class_counts[0])}

X_t, X_test, y_t, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=int(random.random()*1000))
best_params = {'iterations': 432, 'depth': 5, 'learning_rate': 0.028298293648579956, 'random_strength': 9, 'bagging_temperature': 0.08230403615847717, 
               'od_type': 'IncToDec', 'od_wait': 21, 'l2_leaf_reg': 0.1490023617204447, 'border_count': 116, 'leaf_estimation_iterations': 3,
               'leaf_estimation_method': 'Newton', 'bootstrap_type': 'Bayesian'}
model = CatBoostClassifier(**best_params, loss_function='Logloss', eval_metric='Logloss', class_weights=class_weights)
model.fit(X_t, y_t, verbose=0)
y_pred = model.predict_proba(X_test)
balanced_loss = balanced_log_loss(y_test, y_pred,class_weights)
losses.append(balanced_loss)

In [20]:
from sklearn import svm
clf = svm.SVC(kernel='rbf', gamma='scale') # RBF Kernel


In [31]:
losses = []
for i in range(20):
    X_t, X_test, y_t, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=int(random.random()*1000))
    best_params = study.best_params
    #model = CatBoostClassifier(**best_params, loss_function='Logloss', eval_metric='Logloss', class_weights=class_weights)
    model = svm.SVC(kernel='rbf', gamma='scale',probability=True, class_weight='balanced')
    model.fit(X_t, y_t)
    y_pred = model.predict_proba(X_test)
    balanced_loss = balanced_log_loss(y_test, y_pred,class_weights)
    losses.append(balanced_loss)
    

In [32]:
import numpy as np
from scipy import stats

def calculate_confidence_interval(data, confidence=0.95):
    # Convert to numpy array for vectorized operations
    data = np.array(data)
    # Calculate the mean and standard error
    mean = np.mean(data)
    se = stats.sem(data)
    # Calculate the confidence interval
    interval = se * stats.t.ppf((1 + confidence) / 2., len(data)-1)
    return mean - interval, mean + interval

In [33]:
lower, upper = calculate_confidence_interval(losses, confidence=0.95)
print(f'95% confidence interval: ({lower}, {upper})')

95% confidence interval: (0.47466495995974595, 0.504737491626111)
